In [1]:
import pandas as pd
import numpy as np
import os
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.graphics.gofplots import ProbPlot
plt.style.use('seaborn') # pretty matplotlib plots
plt.rc('font', size=14)
plt.rc('figure', titlesize=18)
plt.rc('axes', labelsize=15)

In [2]:
#Use this file path for pc
filepath = r"D:\gradschool\Capstone"
#Use this file path for mac 
# filepath = r"/Users/jacksonjoyner/Desktop/MSBA/Capstone/UseThisData/"
os.chdir(filepath)

In [3]:
playerdf = pd.read_csv("PCATest1.csv")
playerdf = playerdf.rename(columns ={"total cs" : "total_cs"})
playerdf["total_cs"]

0        360
1        155
2        411
3        358
4         37
        ... 
92595    281
92596    208
92597    250
92598    315
92599     41
Name: total_cs, Length: 92600, dtype: int64

Part of this data was incorrect, so I corrected it in excel. For KDA, the calculation is (Kills + Assists)/Deaths. If Deaths is 0, you get an infinite number, so I changed it in those instances to be (kills + Assists) / 1. This is common practice in industry. 

From testing, this is the most accurate model I can create

In [4]:
playerdf

,league,year,split,date,side,playername,position,teamname,kills,deaths,...,D%,earnedgoldshare,damageshare,cspm,damagetakenperminute,earnedgold,earned gpm,PCA15,PCAShare,PCApm
0,EU LCS,2014,Spring,3/28/2014 16:14,Blue,fredy122,top,SK Gaming,0,3,...,0.136364,0.233888,0.266095,8.1509,926.4000,8884,201.1472,1.071165,0.851092,0.192018
1,EU LCS,2014,Spring,3/28/2014 16:14,Blue,Svenskeren,jng,SK Gaming,4,4,...,0.181818,0.168887,0.174089,3.5094,733.4717,6415,145.2453,-1.157427,-0.536872,-1.329622
2,EU LCS,2014,Spring,3/28/2014 16:14,Blue,Jesiz,mid,SK Gaming,1,3,...,0.136364,0.244866,0.284035,9.3057,576.5434,9301,210.5887,1.115521,1.102367,0.530580
3,EU LCS,2014,Spring,3/28/2014 16:14,Blue,CandyPanda,bot,SK Gaming,2,5,...,0.227273,0.254186,0.192568,8.1057,685.9698,9655,218.6038,-0.831752,0.566422,0.338479
4,EU LCS,2014,Spring,3/28/2014 16:14,Blue,nRated,sup,SK Gaming,1,7,...,0.318182,0.098173,0.083214,0.8377,528.8151,3729,84.4302,-2.931761,-1.982099,-2.462070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92595,LCS,2021,Spring,4/3/2021 21:32,Red,Alphari,top,Team Liquid,1,2,...,0.117647,0.220126,0.224882,8.6550,442.2690,7041,216.8686,1.414158,0.404612,0.443739
92596,LCS,2021,Spring,4/3/2021 21:32,Red,Santorin,jng,Team Liquid,1,4,...,0.235294,0.181831,0.093968,6.4066,773.9322,5816,179.1376,0.906147,-0.951782,-0.388794
92597,LCS,2021,Spring,4/3/2021 21:32,Red,Jensen,mid,Team Liquid,3,5,...,0.294118,0.246104,0.327698,7.7002,707.8029,7872,242.4641,1.724921,1.423113,0.463075
92598,LCS,2021,Spring,4/3/2021 21:32,Red,Tactical,bot,Team Liquid,1,3,...,0.176471,0.270988,0.295341,9.7023,284.8460,8668,266.9815,0.601493,1.479895,1.123052


In [5]:
variables = ['kills','deaths','assists','PCA15','PCAShare','PCApm']
x = playerdf[variables].to_numpy() 
y = playerdf["result"]

In [7]:
#https://www.datasklr.com/ols-least-squares-regression/multicollinearity
model = sm.Logit(y, sm.add_constant(x)).fit()
Y_pred = model.predict(sm.add_constant(x))
print_model = model.summary()
print(print_model)

Optimization terminated successfully.
         Current function value: 0.245784
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:                 result   No. Observations:                92600
Model:                          Logit   Df Residuals:                    92593
Method:                           MLE   Df Model:                            6
Date:                Mon, 25 Apr 2022   Pseudo R-squ.:                  0.6454
Time:                        13:37:25   Log-Likelihood:                -22760.
converged:                       True   LL-Null:                       -64185.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.7205      0.031    -55.765      0.000      -1.781      -1.660
x1             0.4857      0.

In [8]:
Y_pred

array([0.01037893, 0.01307817, 0.00948328, ..., 0.00186973, 0.01791716,
       0.04808152])

In [9]:
playerdf["predscore"] = Y_pred
#playerdf.to_csv("LogitModelPreds.csv", index = False)

In [10]:
params = model.params
exp = np.exp(params)
exp

const     0.178976
x1        1.625383
x2        0.408758
x3        1.599312
x4        0.566451
x5        0.098486
x6       17.543619
dtype: float64

In [12]:
logit = sm.Logit(y,sm.add_constant(x)).fit()
print(logit.summary())

Optimization terminated successfully.
         Current function value: 0.245784
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:                 result   No. Observations:                92600
Model:                          Logit   Df Residuals:                    92593
Method:                           MLE   Df Model:                            6
Date:                Mon, 25 Apr 2022   Pseudo R-squ.:                  0.6454
Time:                        13:38:10   Log-Likelihood:                -22760.
converged:                       True   LL-Null:                       -64185.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.7205      0.031    -55.765      0.000      -1.781      -1.660
x1             0.4857      0.

In [13]:
params = logit.params
exp = np.exp(params)
exp

const     0.178976
x1        1.625383
x2        0.408758
x3        1.599312
x4        0.566451
x5        0.098486
x6       17.543619
dtype: float64

In [17]:
conf = logit.conf_int()
conf

,0,1
const,-1.780974,-1.660033
x1,0.471199,0.500287
x2,-0.912249,-0.877017
x3,0.461058,0.478089
x4,-0.601323,-0.535406
x5,-2.369257,-2.266424
x6,2.800245,2.929136


In [22]:
YPred = logit.predict(sm.add_constant(x))
playerdf["predscore"] = YPred

In [24]:
playerdf.to_csv("LogitModelPreds.csv", index = False)